In [1]:
import ray
from ray import rllib, tune
import pandas as pd
import numpy as np
from datetime import datetime
import trading_gym
from trading_gym.registry.gaia.v9.env import GAIAPredictorsContinuousV9
from trading_gym.ray.walkforward import WalkForwardRunner, WalkForwardResults
%matplotlib inline
print(trading_gym.__package__, trading_gym.__version__)
print(ray.__package__, ray.__version__)

/home/Nicholas/trading-gym_0.8.1/venv0.8.1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/Nicholas/trading-gym_0.8.1/venv0.8.1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/Nicholas/trading-gym_0.8.1/venv0.8.1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/Nicholas/trading

trading-gym 0.8.1
ray 0.7.3


In [2]:
# from trading_gym.registry.gaia.v11.env import GAIAPredictorsContinuousV11
from trading_gym.registry.gaia.v12.env import GAIAPredictorsContinuousV12

In [3]:
ray.init()

2019-08-24 23:27:59,360	INFO node.py:498 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-08-24_23-27-59_359363_99481/logs.
2019-08-24 23:27:59,544	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:38993 to respond...
2019-08-24 23:27:59,688	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:33160 to respond...
2019-08-24 23:27:59,693	INFO services.py:809 -- Starting Redis shard with 10.0 GB max memory.
2019-08-24 23:27:59,754	INFO node.py:512 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-08-24_23-27-59_359363_99481/logs.
2019-08-24 23:27:59,757	WARNING services.py:1301 -- Warning: Capping object memory store to 20.0GB. To increase this further, specify `object_store_memory` when calling ray.init() or ray start.
2019-08-24 23:27:59,758	WARNING services.py:1330 -- WARNING: The default object store size of 20.0 GB will use more than 50% of the available memory on this node (29.14 GB). Consider setting the object 

{'node_ip_address': '10.0.5.4',
 'redis_address': '10.0.5.4:38993',
 'object_store_address': '/tmp/ray/session_2019-08-24_23-27-59_359363_99481/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2019-08-24_23-27-59_359363_99481/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2019-08-24_23-27-59_359363_99481'}

## Create the walk-forward partitions
It's responsibility of the user to create the training/test (and maybe validation) partitions to turn the walk forward training. Note that 2-fold split is a particular case of walk-forward training, so you are still able to run a simple 2-fold split.


In [4]:
partitions = list()
for year in range(2007, 2018):
    partition = {
        'training-set': [datetime.min, datetime(year, 12, 31)],
        'test-set': [datetime(year + 1, 1, 1), datetime(year + 1, 12, 31)],
    }
    partitions.append(partition)
partitions


[{'training-set': [datetime.datetime(1, 1, 1, 0, 0),
   datetime.datetime(2007, 12, 31, 0, 0)],
  'test-set': [datetime.datetime(2008, 1, 1, 0, 0),
   datetime.datetime(2008, 12, 31, 0, 0)]},
 {'training-set': [datetime.datetime(1, 1, 1, 0, 0),
   datetime.datetime(2008, 12, 31, 0, 0)],
  'test-set': [datetime.datetime(2009, 1, 1, 0, 0),
   datetime.datetime(2009, 12, 31, 0, 0)]},
 {'training-set': [datetime.datetime(1, 1, 1, 0, 0),
   datetime.datetime(2009, 12, 31, 0, 0)],
  'test-set': [datetime.datetime(2010, 1, 1, 0, 0),
   datetime.datetime(2010, 12, 31, 0, 0)]},
 {'training-set': [datetime.datetime(1, 1, 1, 0, 0),
   datetime.datetime(2010, 12, 31, 0, 0)],
  'test-set': [datetime.datetime(2011, 1, 1, 0, 0),
   datetime.datetime(2011, 12, 31, 0, 0)]},
 {'training-set': [datetime.datetime(1, 1, 1, 0, 0),
   datetime.datetime(2011, 12, 31, 0, 0)],
  'test-set': [datetime.datetime(2012, 1, 1, 0, 0),
   datetime.datetime(2012, 12, 31, 0, 0)]},
 {'training-set': [datetime.datetime(1, 

In [5]:
## Create the config dict
config = ray.rllib.agents.ppo.DEFAULT_CONFIG.copy()
config['env'] = GAIAPredictorsContinuousV12
config['env_config'] = {
    'cost_of_commissions': tune.grid_search([0.00005]),
    'cost_of_spread': 0.0001,
}
config['gamma'] = tune.grid_search([0.95]) # 2 weeks

In [6]:
config['vf_clip_param'] = 0
config['vf_loss_coeff'] = 0
config['lambda'] = 0
config['use_gae'] = True

# need to have vf share layers if lstm is used
config['vf_share_layers'] = True
config['use_lstm']: True
config['batch_mode'] = 'complete_episodes'
config['train_batch_size'] = 4000 # tune.grid_search([4000])
config['sgd_minibatch_size'] = 128
config['num_sgd_iter'] = tune.grid_search([8])
config['entropy_coeff'] =  1e-5 # tune.grid_search([1e-5])
config['kl_coeff'] = 0.2 #tune.grid_search([0.2])
config['kl_target'] = tune.grid_search([0.01])
config['clip_param'] = tune.grid_search([0.8])

config['lr'] = tune.grid_search([1e-5])

# config['model']['custom_model'] = CNN.__name__


In [7]:
# env = GAIAPredictorsContinuousV9()
# length = env.episode_length()

In [8]:
env = GAIAPredictorsContinuousV12()
env.action_space.sample()

/home/Nicholas/trading-gym_0.8.1/venv0.8.1/lib/python3.6/site-packages/trading_gym/transmitter.py:399: FutureWarning:

The 'contains' method is deprecated and will be removed in a future version. Use 'key in index' instead of 'index.contains(key)'



array([0.07634861, 0.02470711])

In [9]:
## Run your walk-forward experiment
walk_forward = WalkForwardRunner(
    env_partitions=partitions,
    trainable=ray.rllib.agents.ppo.PPOTrainer,
    config=config,
    stop={'timesteps_total': 500000},
    checkpoint_freq=1,
)

Note that WalkForwardRunner has constructed the implied ray Experiment(s) from your walk forward settings.

walk_forward.experiments

Note that trials are associated with a `RestoreID`. This `ID` is all you need to restore an agent. Here we are using a grid search of two values for `cost_of_commissions` on two partitions, so we have a total of 4 experiments.

In [10]:
trials = walk_forward.run(verbose=0)
trials

2019-08-24 23:28:03,373	WARNING experiment.py:223 -- All experiments will be using the same SearchAlgorithm.
2019-08-24 23:28:03,415	INFO trial_runner.py:176 -- Starting a new experiment.
2019-08-24 23:28:03,452	WARNING signature.py:108 -- The function with_updates has a **kwargs argument, which is currently not supported.
2019-08-24 23:28:03,471	ERROR log_sync.py:34 -- Log sync requires cluster to be setup with `ray up`.
2019-08-24 23:28:03,564	WARNING util.py:145 -- The `start_trial` operation took 0.12183761596679688 seconds to complete, which may be a performance bottleneck.


(pid=99579) /home/Nicholas/trading-gym_0.8.1/venv0.8.1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=99579)   _np_qint8 = np.dtype([("qint8", np.int8, 1)])
(pid=99579) /home/Nicholas/trading-gym_0.8.1/venv0.8.1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=99579)   _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
(pid=99579) /home/Nicholas/trading-gym_0.8.1/venv0.8.1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=99579)  

(pid=99574) /home/Nicholas/trading-gym_0.8.1/venv0.8.1/lib/python3.6/site-packages/trading_gym/transmitter.py:399: FutureWarning:
(pid=99574) 
(pid=99574) The 'contains' method is deprecated and will be removed in a future version. Use 'key in index' instead of 'index.contains(key)'
(pid=99574) 
(pid=99580) /home/Nicholas/trading-gym_0.8.1/venv0.8.1/lib/python3.6/site-packages/trading_gym/transmitter.py:399: FutureWarning:
(pid=99580) 
(pid=99580) The 'contains' method is deprecated and will be removed in a future version. Use 'key in index' instead of 'index.contains(key)'
(pid=99580) 
(pid=99579) 2019-08-24 23:28:18,473	WARNING util.py:47 -- Install gputil for GPU system monitoring.
(pid=99580) 2019-08-24 23:28:18,868	INFO rollout_worker.py:319 -- Creating policy evaluation worker 1 on CPU (please ignore any CUDA init errors)
(pid=99580) 2019-08-24 23:28:18.892170: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not 

(pid=99580) 2019-08-24 23:28:22,423	INFO rollout_worker.py:485 -- Completed sample batch:
(pid=99580) 
(pid=99580) { 'data': { 'action_prob': np.ndarray((205,), dtype=float32, min=0.0, max=0.159, mean=0.083),
(pid=99580)             'actions': np.ndarray((205, 2), dtype=float32, min=-2.784, max=3.371, mean=-0.039),
(pid=99580)             'advantages': np.ndarray((205,), dtype=float32, min=-2.479, max=2.283, mean=-0.048),
(pid=99580)             'agent_index': np.ndarray((205,), dtype=int64, min=0.0, max=0.0, mean=0.0),
(pid=99580)             'behaviour_logits': np.ndarray((205, 4), dtype=float32, min=-0.01, max=0.009, mean=-0.002),
(pid=99580)             'dones': np.ndarray((205,), dtype=bool, min=0.0, max=1.0, mean=0.024),
(pid=99580)             'eps_id': np.ndarray((205,), dtype=int64, min=444579787.0, max=1742621616.0, mean=975485564.4),
(pid=99580)             'infos': np.ndarray((205,), dtype=object, head={'time': Timestamp('2007-01-25 00:00:00'), 'nlv': 99.99863640908863, 'nr

(pid=99578) /home/Nicholas/trading-gym_0.8.1/venv0.8.1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=99578)   _np_qint8 = np.dtype([("qint8", np.int8, 1)])
(pid=99578) /home/Nicholas/trading-gym_0.8.1/venv0.8.1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=99578)   _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
(pid=99578) /home/Nicholas/trading-gym_0.8.1/venv0.8.1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=99578)  

...

(pid=18723) 2019-08-25 06:58:50,737	WARNING ppo.py:121 -- The magnitude of your environment rewards are more than infx the scale of `vf_clip_param`. This means that it will take more than inf iterations for your value function to converge. If this is not intended, consider increasing `vf_clip_param`.
(pid=18723) 2019-08-25 06:59:09,657	WARNING ppo.py:121 -- The magnitude of your environment rewards are more than infx the scale of `vf_clip_param`. This means that it will take more than inf iterations for your value function to converge. If this is not intended, consider increasing `vf_clip_param`.
(pid=18723) 2019-08-25 06:59:30,024	WARNING ppo.py:121 -- The magnitude of your environment rewards are more than infx the scale of `vf_clip_param`. This means that it will take more than inf iterations for your value function to converge. If this is not intended, consider increasing `vf_clip_param`.
(pid=18723) 2019-08-25 06:59:50,858	WARNING ppo.py:121 -- The magnitude of your environment re

2019-08-25 06:59:54,688	WARNING util.py:145 -- The `process_trial` operation took 0.11229896545410156 seconds to complete, which may be a performance bottleneck.


(pid=18723) 2019-08-25 07:00:11,538	WARNING ppo.py:121 -- The magnitude of your environment rewards are more than infx the scale of `vf_clip_param`. This means that it will take more than inf iterations for your value function to converge. If this is not intended, consider increasing `vf_clip_param`.
(pid=18723) 2019-08-25 07:00:32,857	WARNING ppo.py:121 -- The magnitude of your environment rewards are more than infx the scale of `vf_clip_param`. This means that it will take more than inf iterations for your value function to converge. If this is not intended, consider increasing `vf_clip_param`.
(pid=18723) 2019-08-25 07:00:52,324	WARNING ppo.py:121 -- The magnitude of your environment rewards are more than infx the scale of `vf_clip_param`. This means that it will take more than inf iterations for your value function to converge. If this is not intended, consider increasing `vf_clip_param`.
(pid=18723) 2019-08-25 07:01:11,772	WARNING ppo.py:121 -- The magnitude of your environment re

2019-08-25 07:01:15,860	WARNING util.py:145 -- The `process_trial` operation took 0.1015310287475586 seconds to complete, which may be a performance bottleneck.


[PPO_GAIAPredictorsContinuousV12_restoreID=-5130043545937450704_runID=9fAi6FX2_0_clip_param=0.8,cost_of_commissions=5e-05,gamma=0.95,kl_target=0.01,lr=1e-05,num_sgd_iter=8,
 PPO_GAIAPredictorsContinuousV12_restoreID=-5130043545937450704_runID=9fAi6FX2_0_clip_param=0.8,cost_of_commissions=5e-05,gamma=0.95,kl_target=0.01,lr=1e-05,num_sgd_iter=8,
 PPO_GAIAPredictorsContinuousV12_restoreID=-5130043545937450704_runID=9fAi6FX2_0_clip_param=0.8,cost_of_commissions=5e-05,gamma=0.95,kl_target=0.01,lr=1e-05,num_sgd_iter=8,
 PPO_GAIAPredictorsContinuousV12_restoreID=-5130043545937450704_runID=9fAi6FX2_0_clip_param=0.8,cost_of_commissions=5e-05,gamma=0.95,kl_target=0.01,lr=1e-05,num_sgd_iter=8,
 PPO_GAIAPredictorsContinuousV12_restoreID=-5130043545937450704_runID=9fAi6FX2_0_clip_param=0.8,cost_of_commissions=5e-05,gamma=0.95,kl_target=0.01,lr=1e-05,num_sgd_iter=8,
 PPO_GAIAPredictorsContinuousV12_restoreID=-5130043545937450704_runID=9fAi6FX2_0_clip_param=0.8,cost_of_commissions=5e-05,gamma=0.95,kl

In [ ]:
# stop

# Temp

In [ ]:
agent = ray.rllib.agents.ppo.PPOTrainer(config, GAIAPredictorsContinuousV11)

In [ ]:
env = GAIAPredictorsContinuousV9()
state = env.reset()
state

In [ ]:
agent.compute_action(state)

# Restore

In [11]:
results = WalkForwardResults('/home/Nicholas/trading-gym_0.8.1/trading-gym/notebooks/registry/gaia/v9/logs')
results

WalkForwardResults(['GAIAPredictorsContinuousV11', 'GAIAPredictorsContinuousV12'])

In [12]:
env_results = results['GAIAPredictorsContinuousV12']
env_results

EnvResults(GAIAPredictorsContinuousV12)

In [13]:
# Step 1.
env = env_results.make_env(
    env_config={
        'cost_of_commissions': 0.00005,
        'cost_of_spread': 0.0001,
        'folds': {
            'training-set': [datetime.min, datetime(2008, 3, 18)],
            'test-set': [datetime(2008, 3, 19), datetime.max],
        }
    },
)

In [14]:
env_results.restore_ids

{-5130043545937450704: [AgentResults(GAIAPredictorsContinuousV12_1-01-01_to_2013-12-31/PPO_GAIAPredictorsContinuousV12_restoreID=-5130043545937450704_runID=9fAi6FX2_0_clip_param=0.8,cost_of_commissions=5e-05,gamma=0.9_2019-08-25_03-39-30ra635rbb),
  AgentResults(GAIAPredictorsContinuousV12_1-01-01_to_2009-12-31/PPO_GAIAPredictorsContinuousV12_restoreID=-5130043545937450704_runID=9fAi6FX2_0_clip_param=0.8,cost_of_commissions=5e-05,gamma=0.9_2019-08-25_00-56-31q62e48jc),
  AgentResults(GAIAPredictorsContinuousV12_1-01-01_to_2007-12-31/PPO_GAIAPredictorsContinuousV12_restoreID=-5130043545937450704_runID=9fAi6FX2_0_clip_param=0.8,cost_of_commissions=5e-05,gamma=0.9_2019-08-24_23-28-033mm0loo1),
  AgentResults(GAIAPredictorsContinuousV12_1-01-01_to_2008-12-31/PPO_GAIAPredictorsContinuousV12_restoreID=-5130043545937450704_runID=9fAi6FX2_0_clip_param=0.8,cost_of_commissions=5e-05,gamma=0.9_2019-08-25_00-15-356hob7ou4),
  AgentResults(GAIAPredictorsContinuousV12_1-01-01_to_2012-12-31/PPO_GAIAP

In [16]:
2557338427780778455
print(list(env_results.restore_ids)[-1])

-8151400312683044895


In [15]:
restore_id = list(env_results.restore_ids)[-1]
restore_id= -5130043545937450704

In [16]:
policy = env_results.make_policy(
    env=env,
    restore_id=restore_id,
    checkpoint_nr=None,  # use None (or don't specify) to use last checkpoint available
)
policy

In [17]:
history = policy.history()
# history

In [18]:
# Step 3.
episode = env.sample_episode(fold='test-set', policy=policy, verbose=False)

2019-08-25 10:52:49,911	INFO rollout_worker.py:319 -- Creating policy evaluation worker 0 on CPU (please ignore any CUDA init errors)
2019-08-25 10:52:50,148	INFO dynamic_tf_policy.py:324 -- Initializing loss function with dummy input:

{ 'action_prob': <tf.Tensor 'default_policy/action_prob:0' shape=(?,) dtype=float32>,
  'actions': <tf.Tensor 'default_policy/actions:0' shape=(?, 2) dtype=float32>,
  'advantages': <tf.Tensor 'default_policy/advantages:0' shape=(?,) dtype=float32>,
  'behaviour_logits': <tf.Tensor 'default_policy/behaviour_logits:0' shape=(?, 4) dtype=float32>,
  'dones': <tf.Tensor 'default_policy/dones:0' shape=(?,) dtype=bool>,
  'new_obs': <tf.Tensor 'default_policy/new_obs:0' shape=(?, 5) dtype=float32>,
  'obs': <tf.Tensor 'default_policy/observation:0' shape=(?, 5) dtype=float32>,
  'prev_actions': <tf.Tensor 'default_policy/action:0' shape=(?, 2) dtype=float32>,
  'prev_rewards': <tf.Tensor 'default_policy/prev_reward:0' shape=(?,) dtype=float32>,
  'rewards': 

(pid=38401) /home/Nicholas/trading-gym_0.8.1/venv0.8.1/lib/python3.6/site-packages/trading_gym/transmitter.py:399: FutureWarning:
(pid=38401) 
(pid=38401) The 'contains' method is deprecated and will be removed in a future version. Use 'key in index' instead of 'index.contains(key)'
(pid=38401) 
(pid=38541) /home/Nicholas/trading-gym_0.8.1/venv0.8.1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=38541)   _np_qint8 = np.dtype([("qint8", np.int8, 1)])
(pid=38541) /home/Nicholas/trading-gym_0.8.1/venv0.8.1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=38541)   _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
(pid

2019-08-25 10:53:01,137	INFO trainable.py:105 -- _setup took 12.787 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2019-08-25 10:53:01,140	WARNING util.py:47 -- Install gputil for GPU system monitoring.
2019-08-25 10:53:01,216	INFO tf_run_builder.py:92 -- Executing TF run without tracing. To dump TF timeline traces to disk, set the TF_TIMELINE_DIR environment variable.


(pid=38541) 2019-08-25 10:53:01,305	INFO rollout_worker.py:319 -- Creating policy evaluation worker 2 on CPU (please ignore any CUDA init errors)
(pid=38541) 2019-08-25 10:53:01.322587: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
(pid=38541) /home/Nicholas/trading-gym_0.8.1/venv0.8.1/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning:
(pid=38541) 
(pid=38541) Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
(pid=38541) 


2019-08-25 10:53:04,725	INFO rollout_worker.py:319 -- Creating policy evaluation worker 0 on CPU (please ignore any CUDA init errors)
2019-08-25 10:53:06,920	INFO rollout_worker.py:742 -- Built policy map: {'default_policy': <ray.rllib.policy.tf_policy_template.PPOTFPolicy object at 0x7f3a6c745c50>}
2019-08-25 10:53:06,921	INFO rollout_worker.py:743 -- Built preprocessor map: {'default_policy': <ray.rllib.models.preprocessors.NoPreprocessor object at 0x7f3a6c745828>}
2019-08-25 10:53:06,926	INFO rollout_worker.py:356 -- Built filter map: {'default_policy': <ray.rllib.utils.filter.NoFilter object at 0x7f3a6c7455f8>}
2019-08-25 10:53:06,988	INFO multi_gpu_optimizer.py:93 -- LocalMultiGPUOptimizer devices ['/cpu:0']


...

2019-08-25 10:53:30,798	INFO rollout_worker.py:319 -- Creating policy evaluation worker 0 on CPU (please ignore any CUDA init errors)
2019-08-25 10:53:32,765	INFO rollout_worker.py:742 -- Built policy map: {'default_policy': <ray.rllib.policy.tf_policy_template.PPOTFPolicy object at 0x7f3a26dd2b38>}
2019-08-25 10:53:32,766	INFO rollout_worker.py:743 -- Built preprocessor map: {'default_policy': <ray.rllib.models.preprocessors.NoPreprocessor object at 0x7f3a26dd2710>}
2019-08-25 10:53:32,770	INFO rollout_worker.py:356 -- Built filter map: {'default_policy': <ray.rllib.utils.filter.NoFilter object at 0x7f3a26dd2550>}
2019-08-25 10:53:32,818	INFO multi_gpu_optimizer.py:93 -- LocalMultiGPUOptimizer devices ['/cpu:0']


(pid=39335) /home/Nicholas/trading-gym_0.8.1/venv0.8.1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=39335)   _np_qint8 = np.dtype([("qint8", np.int8, 1)])
(pid=39335) /home/Nicholas/trading-gym_0.8.1/venv0.8.1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=39335)   _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
(pid=39335) /home/Nicholas/trading-gym_0.8.1/venv0.8.1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=39335)  

2019-08-25 10:53:41,015	INFO trainable.py:105 -- _setup took 11.345 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2019-08-25 10:53:41,019	WARNING util.py:47 -- Install gputil for GPU system monitoring.


(pid=39335) 2019-08-25 10:53:40,889	INFO rollout_worker.py:319 -- Creating policy evaluation worker 1 on CPU (please ignore any CUDA init errors)
(pid=39335) 2019-08-25 10:53:40.912704: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
(pid=39336) 2019-08-25 10:53:41,062	INFO rollout_worker.py:319 -- Creating policy evaluation worker 2 on CPU (please ignore any CUDA init errors)
(pid=39336) 2019-08-25 10:53:41.085046: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
(pid=39335) 2019-08-25 10:53:41,224	INFO dynamic_tf_policy.py:324 -- Initializing loss function with dummy input:
(pid=39335) 
(pid=39335) { 'action_prob': <tf.Tensor 'default_policy/action_prob:0' shape=(?,) dtype=float32>,
(pid=39335)   'actions': <tf.Tensor 'default_policy/actions:0' shape=(?, 2) dtype=float32>,
(pid=39335)   'adva

2019-08-25 10:53:43,816	INFO rollout_worker.py:319 -- Creating policy evaluation worker 0 on CPU (please ignore any CUDA init errors)
2019-08-25 10:53:45,678	INFO rollout_worker.py:742 -- Built policy map: {'default_policy': <ray.rllib.policy.tf_policy_template.PPOTFPolicy object at 0x7f3a120cf278>}
2019-08-25 10:53:45,679	INFO rollout_worker.py:743 -- Built preprocessor map: {'default_policy': <ray.rllib.models.preprocessors.NoPreprocessor object at 0x7f3a12175e10>}
2019-08-25 10:53:45,682	INFO rollout_worker.py:356 -- Built filter map: {'default_policy': <ray.rllib.utils.filter.NoFilter object at 0x7f3a27b632e8>}
2019-08-25 10:53:45,723	INFO multi_gpu_optimizer.py:93 -- LocalMultiGPUOptimizer devices ['/cpu:0']
2019-08-25 10:53:50,712	WARNING util.py:47 -- Install gputil for GPU system monitoring.
2019-08-25 10:53:53,514	INFO rollout_worker.py:319 -- Creating policy evaluation worker 0 on CPU (please ignore any CUDA init errors)
2019-08-25 10:53:55,315	INFO rollout_worker.py:742 -- B

In [19]:
episode.renderer.cumulative_performance.to_plotly()
episode.renderer.target_weights.to_plotly()
episode.renderer.annual_returns.to_plotly()
episode.renderer.tearsheet()

Strategy  \
Context              From                              2008-03-19   
                     To                                2018-08-28   
                     Years                                10.4493   
                     Observations                            2725   
                     Risk-free asset        Index(USD 1M Deposit)   
                     Risk-free CAGR                    0.00681294   
Return               CAGR                               0.0902611   
                     CAGR over cash                     0.0834481   
                     Overall return                       1.46699   
Risk                 Volatility                          0.103097   
                     Downside volatility                0.0843604   
                     Upside volatility                  0.0704598   
                     Max drawdown                       -0.228944   
                     Martin risk                        0.0711368   
                     VaR 5%                           -0.00894882   
                     VaR 2%                            -0.0136161   
                     Expected shortfall 5%             -0.0151012   
                     Expected shortfall 2%             -0.0211425   
Risk-adjusted return Sharpe ratio                        0.809414   
                     Sortino ratio                       0.989187   
                     Calmar ratio                        0.364492   
                     Martin ratio                         1.17307   
Outperformance       Benchmark name         Index(Aric-Benchmark)   
                     CAGR over benchmark               -0.0683245   
                     Information ratio                  -0.627661   
                     CAPM Alpha                          0.021317   
                     CAPM Beta                           0.435876   
Weights              Cash(USD)                         0.00374298   
                     ETF(Russell 1000)                   0.364757   
                     ETF(7-10Y T-Bills)                    0.6315   
                     Leverage mean                              1   
                     Turnover daily                     0.0120442   
                     Turnover annual                      3.03514   

                                            Index(Aric-Benchmark)  \
Context              From                              2008-03-19   
                     To                                2018-08-28   
                     Years                                10.4493   
                     Observations                            2725   
                     Risk-free asset        Index(USD 1M Deposit)   
                     Risk-free CAGR                    0.00681294   
Return               CAGR                                0.158586   
                     CAGR over cash                      0.151773   
                     Overall return                       3.65592   
Risk                 Volatility                         0.0970738   
                     Downside volatility                 0.069906   
                     Upside volatility                  0.0705399   
                     Max drawdown                      -0.0865477   
                     Martin risk                        0.0189993   
                     VaR 5%                           -0.00895352   
                     VaR 2%                            -0.0134313   
                     Expected shortfall 5%             -0.0137858   
                     Expected shortfall 2%              -0.017882   
Risk-adjusted return Sharpe ratio                         1.56348   
                     Sortino ratio                         2.1711   
                     Calmar ratio                         1.75363   
                     Martin ratio                         7.98834   
Outperformance       Benchmark name         Index(Aric-Benchmark)   
                     CAGR over benchmark              

In [20]:
# cost2restore_id = {
#     0.00001: -3264367374635941251,    
# }

In [21]:
# nr2episode = dict()
# for cost_of_commissions, restore_id in cost2restore_id.items():
#     nr2episode[cost_of_commissions] = env_results.get_nr2episode(
#         restore_id=restore_id,
#         checkpoint_nrs=np.arange(1, 126, 1),
#         fold='test-set',
#         env_config={
#             'folds': {
#                 'training-set': [datetime.min, datetime(2008, 3, 18)],
#                 'test-set': [datetime(2008, 3, 19), datetime.max],
#             }
#         }
#     )

In [ ]:
cost_of_commissions = 0.00001
nr2episode[cost_of_commissions].plot_weights()

In [ ]:
nr2episode[cost_of_commissions].plot_levels()

In [ ]:
nr2episode[cost_of_commissions].plot_metrics_as_we_train()